# Hospitalization cost variation master file

Date created: 1/23/23 <br>
Last updated: 1/23/23 <br>
Adapted from: healthrex_ml materials by Conor Corbin ([202301118_healthrex_ml_workshop](https://github.com/HealthRex/healthrex_ml/tree/main/examples/20230118_healthrex_ml_workshop))

**Table of Contents [Tentative]** <br>
0 Inputs and setup <br>
1 Cohort selection <br>
2 Feature extraction <br>
3 Preliminary data visualization <br>
4 Analysis <br>

Additional: 2.5 Data loading, feature selection, model evaluation (part of analysis?)

## 0 Inputs and setup
### 0.1 Global variables
Update for your project

In [ ]:
# Your local home directory
user_id = 'selinapi'

# Source data projects and datasets
nero_gcp_project = 'som-nero-phi-jonc101-secure' # *** Label rest of these
cdm_project_id = 'som-nero-phi-jonc101'
cdm_dataset_id = 'shc_core_2021'

# NERO project and dataset where you are saving your data
work_project_id = nero_gcp_project
work_dataset_id = 'proj_IP_variation'

# Cohort dataset name
cohort_id = 'cohort_drg_221'

# Hours after admission date to set index time
index_lag = 24 # NOTE: lag is from admission DATE (midnight) rather than admission TIME yet as of 1/23/23

# Control variables to run sections of code
run_cohortselection = 0 # Last run: 1/29/23
run_cohortchecks = 1
run_featurizer = 1 # Last run: 1/28/23

## Setup environment / credentials

In [ ]:
from google.cloud import bigquery
import os
import pandas as pd
import sys
import yaml
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
# GCP credentials for Mac: Ran steps linked here to create JSON credentials and file path (https://github.com/HealthRex/CDSS/blob/master/scripts/DevWorkshop/ReadMe.GoogleCloud-BigQuery-VPC.txt)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = (
    f'/Users/{user_id}/.config/gcloud/application_default_credentials.json'
)
os.environ['GCLOUD_PROJECT'] = nero_gcp_project

# Instantiate a client object so you can make queries
client = bigquery.Client()

#  Create a dataset in project to write all our tables there (if it does not exist already)
client.create_dataset(f"{work_project_id}.{work_dataset_id}", exists_ok=True)

## 1 Cohort selection and outcome calculation

### 1.1 Define cohort of admissions based on DRG code
First pass: Creates dataset of admissions with APR-DRG of 221, 245, and 247 with the following columns (1/29/23 - See additions to DRG codes in code comments below)
1. anon_id : id of the patient 
2. observation_id : id of the ML example (observation)

Also Merges outcome variable.
Currently using direct cost; note: merges identifying data, be careful!! *** Add cost breakdowns later
Note: true date from shc_map_2021 in highest security dataset + jitter = anonymized date in lower security dataset


In [ ]:
# Create dataset of gastrointestinal (GI) admissions with APR-DRG of 221, 245, and 247 with the following columns 
# *** Think of how to make this adaptable for different DRGs, or make cohort creation its own class that you call (like for Healthrex_ML)
# *** ASSUMPTION: sum costs that occur within the dates of the IP admission; ALTERNATE: Only sum costs where Inpatient_C == 'I'
# 1/29/23 UPDATE: Removed 221 and added
#     230 (MAJOR SMALL BOWEL PROCEDURES, drg_id = 6267, code set 3)
#     231 (MAJOR LARGE BOWEL PROCEDURES, drg_id = 6268, code set 3)

#     Also found the following, but did not add because vague or led to duplicate admissions (some admissions were coded with multiple DRG systems)
#     254 (OTHER DIGESTIVE SYSTEM DIAGNOSES, drg_id = 2447) 
#     330 (MAJOR SMALL AND LARGE BOWEL PROCEDURES WITH CC, drg_id = 1836, code set 6)
#     329 (MAJOR SMALL AND LARGE BOWEL PROCEDURES WITH MCC, drg_id = 1835, code set 6)
#     because DRG 221 was not being captured in the cost data. 
# *** Think of ways to capture bowel procedures more effectively/in automated fashion, while being comprehensive and not introducing redundancies from admissions coded with multiple systems
# *** Are there any patients with 221 but NOT 230 or 231? (Context: Admissions were coded with both 221 and either 230 or 231, so I had to remove these duplicates in this first pass of the analysis)
query= """
    CREATE OR REPLACE TABLE
    `{work_project_id}.{work_dataset_id}.{cohort_id}` AS
    
    --Get anonymized admission DRG details
    WITH 
    DRG_adms AS 
    (
    SELECT DISTINCT
        a.anon_id, 
        a.pat_enc_csn_id_jittered as observation_id, 
        timestamp(date_add(CAST(a.hosp_adm_date_jittered as datetime), interval {index_lag} hour)) as index_time,
        a.hosp_adm_date_jittered as adm_date,
        a.hosp_disch_date_jittered as disch_date,
        TIMESTAMP_DIFF(a.hosp_disch_date_jittered, a.hosp_adm_date_jittered, DAY) + 1 as LOS,
        b.drg_mpi_code,
        b.drg_id,
        b.drg_name,
        b.DRG_CODE_SET_C
    FROM `{cdm_project_id}.shc_core_2021.f_ip_hsp_admission` a
    LEFT JOIN `{cdm_project_id}.{cdm_dataset_id}.drg_code` b
    ON a.anon_id = b.anon_id AND a.pat_enc_csn_id_jittered = b.pat_enc_csn_id_coded
    WHERE (b.drg_mpi_code IN ('245', '247') AND b.drg_id LIKE '2%')
        OR (b.drg_mpi_code IN ('230', '231') AND b.drg_id LIKE '626%')
    ),

    --Link costs to anonymized ID
    SHC_costs AS
    (
    SELECT 
        b.anon_id,
        a.AdmitDate + b.jitter as adm_date_jittered,
        a.DischargeDate + b.jitter as disch_date_jittered,
        --a.VisitCount,
        a.MSDRGWeight,
        --a.Inpatient_C,
        --a.ServiceCategory_C,
        a.Cost_Direct
    FROM `{nero_gcp_project}.shc_cost.costUB` a
    LEFT JOIN `{nero_gcp_project}.starr_map.shc_map_2021` b
    ON cast(a.mrn AS string) = b.mrn
    )

    --Join admission DRG details and costs by patient ID and overlapping dates (NOTE: manually add all cost variables you want to keep)
    SELECT DISTINCT
        a.*,
        SUM(b.Cost_Direct) OVER(PARTITION BY a.observation_id) AS Cost_Direct,
        MAX(b.MSDRGWeight) OVER(PARTITION BY a.observation_id) AS MSDRGWeight
    FROM DRG_adms a
    LEFT JOIN SHC_costs b
    --ON a.anon_id = b.anon_id AND a.adm_date <= b.disch_date_jittered AND a.disch_date >= b.adm_date_jittered --Join by overlapping dates
    ON a.anon_id = b.anon_id AND a.adm_date <= b.adm_date_jittered AND b.disch_date_jittered <= a.disch_date --Join if cost dates are within IP admission dates
""".format_map({'cdm_project_id': cdm_project_id,
                'cdm_dataset_id': cdm_dataset_id,
                'nero_gcp_project': nero_gcp_project,
                'work_project_id': work_project_id,
                'work_dataset_id': work_dataset_id,
               'cohort_id': cohort_id,
               'index_lag': index_lag})

if run_cohortselection == 1:
    client.query(query).result();

### 1.2 Cohort explorations and sanity checks
Record of tests and checks

In [ ]:
# Download cohort data temporarily for checks
if run_cohortchecks == 1:
    query = """
        SELECT 
            *
        FROM `{work_project_id}.{work_dataset_id}.{cohort_id}`
    """.format_map({'work_project_id': work_project_id,
                    'work_dataset_id': work_dataset_id,
                   'cohort_id': cohort_id})

    df = client.query(query).to_dataframe()

In [ ]:
if run_cohortchecks == 1:
    # Cohort size
    print(df.shape) # 7428, 15. 1/29/23 Update: 6232
    print(df["observation_id"].nunique())
    print(type(df))
    
    # Duplicate IP admissions?
    dups = df[df.duplicated(subset=['observation_id'], keep=False)].sort_values(by=['anon_id', 'adm_date'])
    print(dups)

    # Missing values
    print(df.isna().sum())
    
    # Costs: are there non-inpatient costs that overlap with an IP visit?

    # Number of IP admissions with costs

## 2 Feature extraction

### 2.1 Define a set of extractors

From Conor Corbin's Healthrex_ML API; extractor definitions [here]()

In [ ]:
if run_featurizer == 1:
    from healthrex_ml.extractors import (
        AgeExtractor,
        RaceExtractor,
        SexExtractor,
        EthnicityExtractor,
        ProcedureExtractor,
        PatientProblemExtractor,
        MedicationExtractor,
        LabOrderExtractor,
        LabResultBinsExtractor,
        FlowsheetBinsExtractor
    )

    USED_EXTRACTORS = [AgeExtractor,
        RaceExtractor,
        SexExtractor,
        EthnicityExtractor,
        ProcedureExtractor,
        PatientProblemExtractor,
        MedicationExtractor,
        LabOrderExtractor,
        LabResultBinsExtractor,
        FlowsheetBinsExtractor
    ]

    cohort_table=f"{work_project_id}.{work_dataset_id}.{cohort_id}"
    feature_table=f"{work_project_id}.{work_dataset_id}.{cohort_id}_feature_matrix"
    extractors = [
        ext(cohort_table_id=cohort_table, feature_table_id=feature_table)
        for ext in USED_EXTRACTORS
    ]

### 2.2 Define a featurizer and create a feature matrix

Will execute a series of SQL queries defined by the extractors to build up a long form feature matrix and save to bigquery. Additionally, will read in the long form feature matrix and build up a sparse (CSR) matrix without doing the expensive pivot operation.  Will save locally. Automatically generates train/test split by using last year of data as test set.  Can use `train_years` and `test_years` arguments in the `__init__` function to modify. 

Implementation of [BagOfWordsFeaturizer](https://github.com/HealthRex/healthrex_ml/blob/main/healthrex_ml/featurizers/starr_featurizers.py#L239)

In [ ]:
if run_featurizer == 1:
    from healthrex_ml.featurizers import BagOfWordsFeaturizer

    featurizer = BagOfWordsFeaturizer(  cohort_table_id   = cohort_table,
                                        feature_table_id  = feature_table,
                                        extractors        = extractors,
                                        outpath           = f"./{cohort_id}_artifacts",
                                        tfidf             = True
                                )

    featurizer()

## 3 Load and visualize data

### 3.1 Read in data

In [ ]:
from sklearn.linear_model import LinearRegression
from scipy.sparse import load_npz
from scipy.sparse.linalg import lsqr

# Read in features
features = pd.read_csv(os.path.join(f"./{cohort_id}_artifacts/feature_order.csv"))

# Read in train data
X_train_full = load_npz(os.path.join(f"./{cohort_id}_artifacts/train_features.npz"))
y_train_full = pd.read_csv(os.path.join(f"./{cohort_id}_artifacts/train_labels.csv"))

# Remove any rows with missing labels (for censoring tasks)
task = 'Cost_Direct' # **** Make a global var?
observed_inds_train = y_train_full[~y_train_full[task].isnull()].index
X_train = X_train_full[observed_inds_train]
y_train = y_train_full.iloc[observed_inds_train].reset_index()
y_train = y_train[task]

In [ ]:
# Read in test data
X_test_full = load_npz(os.path.join(f"./{cohort_id}_artifacts/test_features.npz"))
y_test_full = pd.read_csv(os.path.join(f"./{cohort_id}_artifacts/test_labels.csv"))

# Remove any rows with missing labels (for censoring tasks)
task = 'Cost_Direct' # **** Make a global var?
observed_inds_test = y_test_full[~y_test_full[task].isnull()].index
X_test = X_test_full[observed_inds_test]
y_test = y_test_full.iloc[observed_inds_test].reset_index()
y_test = y_test[task]

In [ ]:
# Create full datasets
# Reference: https://cmdlinetips.com/2019/07/how-to-slice-rows-and-columns-of-sparse-matrix-in-python/
drg_los_outcomes = y_train_full.append(y_test_full)
y_full = y_train.append(y_test)
len(y_full)



### 3.2 Baseline characteristics (IN DEVELOPMENT)

In [ ]:
# Demographics
demog = features[features['features'].str.startswith(('race_', 'sex_', 'Age_', 'eth'))]
demog = demog.sort_values(by='features')
print(demog)



from scipy import sparse
# X_full = vstack(X_train_full, X_test_full)
X_demog = X_train_full.tocsr()[:,demog['indices']].todense()
X_demog = np.concatenate((X_demog, X_test_full.tocsr()[:,demog['indices']].todense()))
# X_demog_summary = 

# All data, data with costs, training data, test data

In [ ]:
X_full

### 3.3 Cost visualizations

In [ ]:
# Histogram of all costs
plt.hist(y_full)

In [ ]:
# Histogram of costs within Xth percentile
percentile = 90
index = math.floor((percentile / 100)*len(y_full))
y_subset = y_full.sort_values()
y_subset = y_subset[:index]
plt.hist(y_subset)

In [ ]:
# LOS vs. costs - pretty linear correlation, with some outliers?
plt.scatter(drg_los_outcomes['LOS'], drg_los_outcomes['Cost_Direct'], alpha=0.2)

In [ ]:
drg_los_outcomes

In [ ]:
# LOS vs. costs - stratified by DRG
# How to: https://stackoverflow.com/questions/21654635/scatter-plots-in-pandas-pyplot-how-to-plot-by-category
# Also: https://www.statology.org/matplotlib-scatterplot-color-by-value/
# **** 1/29/23 This was how I found out that there aren't really any major small & large bowel procedures coded DRG 221 in the cost data

import matplotlib.pyplot as plt
groups = drg_los_outcomes.groupby('drg_name')
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group.LOS, group.Cost_Direct, marker='o', linestyle='', ms=3, label=name, alpha=0.3)
ax.legend()


plt.show()

In [ ]:
# Histogram of cost per day

## 4 Analysis

### 4.2 Linear regression
Sparse linear regression: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.lsqr.html

In [ ]:
x, istop, itn, r1norm, r2norm = lsqr(X_train, y_train)[:5]

# **** Note: system is under-determined (many more features than data points), so there is a near perfect fit....

In [ ]:
y_hat = X_train@x
print(max(abs(y_hat - y_train)))
print(sum(abs(y_hat - y_train))/len(y_train))
print(r2norm)

In [ ]:
y_hat_test = X_test@x
print(max(abs(y_hat_test - y_test)))
print(sum(abs(y_hat_test - y_test))/len(y_test))
# print(r2norm)
# *** Find regression evaluation approach robust to outliers

### 4.3 LASSO
Read more here: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train, y_train)

print(clf.coef_)

print(clf.intercept_)

In [ ]:
# Try increasing tolerance and iterations
clf2 = linear_model.Lasso(alpha=0.1, max_iter=5000, tol=0.01, selection='random')
clf2.fit(X_train, y_train)

print(clf2.coef_)

print(clf2.intercept_)

In [ ]:
sum(clf2.coef_ == 0)/len(clf2.coef_) # OK, ~90% of features are 0

In [ ]:
# Bind features and coefficients
feature_coefs = features.copy()
feature_coefs['coefs'] = clf2.coef_.tolist()
feature_coefs

In [ ]:
# Features with nonzero LASSO coefficients
nonzero_features = feature_coefs[feature_coefs['coefs'] != 0]
nonzero_features

In [ ]:
# Evaluate training accuracy by percentage error
y_hat = clf2.predict(X_train)
pct_error = abs((y_hat - y_train)/y_train)
min(pct_error), max(pct_error), sum(pct_error)/len(pct_error)

In [ ]:
# Plot absolute training error
import matplotlib.pyplot as plt
plt.scatter(y_train, pct_error, alpha = 0.2)
plt.xlabel('True cost (USD)')
plt.ylabel('Absolute percentage error')
plt.show()

In [ ]:
# Plot absolute training error (log-log)
plt.scatter(y_train, pct_error, alpha = 0.2)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('True cost (USD, log scale)')
plt.ylabel('Absolute percentage error (log scale)')
plt.show()

In [ ]:
# Evaluate test accuracy by percentage error
y_hat_test = clf2.predict(X_test)
pct_error_test = abs((y_hat_test - y_test)/y_test)
min(pct_error_test), max(pct_error_test), sum(pct_error_test)/len(pct_error_test)

In [ ]:
# Plot absolute testing error
plt.scatter(y_test, pct_error_test, alpha = 0.2)
plt.xlabel('True cost (USD)')
plt.ylabel('Absolute percentage error')
plt.show()

In [ ]:
# Plot absolute testing error (log-log)
plt.scatter(y_test, pct_error_test, alpha = 0.2)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('True cost (USD, log scale)')
plt.ylabel('Absolute percentage error (log scale)')
plt.show()

### Model evaluation (NOT USED - From Conor's healthrex_ml example program)

### Train a set of gradient boosted trees

Implementation of [LightGBMTrainer](https://github.com/HealthRex/healthrex_ml/blob/main/healthrex_ml/trainers/sklearn_trainers.py#L23)

In [ ]:
from healthrex_ml.trainers import LightGBMTrainer # SP 1/18/23 Grace replaced with BaselineModelTrainer (uses random forest, since LightGBMTrainer was causing a segmentation fault)

trainer = LightGBMTrainer(working_dir=f"./{cohort_id}_artifacts")
tasks = ['Cost_Direct']

for task in tasks:
    trainer(task)

### Evaluate model performance on test set and dump 

Implementation of [BinaryEvaluator](https://github.com/HealthRex/healthrex_ml/blob/main/healthrex_ml/evaluators/evaluators.py#L21) 

In [ ]:
from healthrex_ml.evaluators import BinaryEvaluator
from tqdm import tqdm

for task in tqdm(tasks):
    evalr = BinaryEvaluator(
        outdir=f"./{RUN_NAME}_artifacts/{task}_performance_artificats/",
        task_name=task
    )
    df_yhats = pd.read_csv(os.path.join(trainer.working_dir, f"{task}_yhats.csv"))
    evalr(df_yhats.labels, df_yhats.predictions)